In [1]:
!pip install thefuzz

In [2]:
import pandas as pd
import sqlite3
import re
from thefuzz import fuzz

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
conn = sqlite3.connect("./melee_player_database.db")

In [4]:
players = pd.read_sql('SELECT * FROM players', conn)

In [5]:
sets = pd.read_sql('SELECT * FROM sets', conn)

In [6]:
tournament_info = pd.read_sql('SELECT * FROM tournament_info', conn)

In [7]:
players

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,-Caius-,-Caius-,"[""-Caius-""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 2...","""""",None
1,melee,1028,Wizzrobe,"[""Wizzzrobe"", ""COG | Wizzrobe"", ""COG Wizzrobe""...","[""Envy"", ""ExG""]","{""twitter"": [""Wizzrobe""]}",United States,FL,,US,CA,Los Angeles,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 4...","{""melee/captainfalcon"": 659}",None
2,melee,1032,Bladewise,"[""Bladewise00"", ""[62-Bit] Bladewise"", ""Bladewi...","[""W2W""]","{""twitter"": [""bladewise00""]}",United States,WA,,US,CA,Seattle,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 9...","{""melee/peach"": 115}",None
3,melee,GOL_482_JSalt,GOL_482_JSalt,"[""GOL_482_JSalt""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 1...","""""",None
4,melee,RiSK__,RiSK__,"[""RiSK__""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 2...","""""",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45537,melee,2263389,Autumn from 207,"[""Autumn from 207""]",[],"{""twitter"": []}",United States,None,None,None,None,None,"[{""key"": ""the-foxy-horror-picture-show-a-gayle...","{""melee/falco"": 4, ""melee/marth"": 2}",None
45538,melee,21721,BasedVapeDuck,"[""BasedVapeDuck""]",[],"{""twitter"": []}",United States,MA,None,None,None,None,"[{""key"": ""kalvar-cup-26-the-scary-one__melee-s...","{""melee/ganondorf"": 8}",None
45539,melee,185294,Tron Bonne,"[""Tron Bonne""]","[""MYC""]","{""twitter"": []}",United States,KS,None,US,KS,Lawrence,"[{""key"": ""rock-chalk-jab-lock-85__melee-single...","""""",None
45540,melee,2512542,a bottle of soap,"[""a bottle of soap""]",[],"{""twitter"": []}",United States,FL,None,None,None,None,"[{""key"": ""scss-46__scss-singles"", ""placing"": 6...","{""melee/jigglypuff"": 12}",None


In [8]:
sets

,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data
0,,melee,mlg-world-finals-2015,1004,1004,1003,3,1,"[""Winners Top 32 Round 5"", ""Winners Top 32 Rou...",,,0,NaN,[]
1,,melee,mlg-world-finals-2015,1004,1004,16342,3,1,"[""Winners Top 32 Round 4"", ""Winners Top 32 Rou...",,,1,NaN,[]
2,,melee,mlg-world-finals-2015,1004,1004,4442,3,0,"[""Winners Top 32 Round 3"", ""Winners Top 32 Rou...",,,2,NaN,[]
3,,melee,mlg-world-finals-2015,16342,16342,1003,3,2,"[""Winners Top 32 Round 3"", ""Winners Top 32 Rou...",,,3,NaN,[]
4,,melee,mlg-world-finals-2015,1004,1004,Prince_Abu,3,0,"[""Winners Top 32 Round 2"", ""Winners Top 32 Rou...",,,4,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456434,gg__41949154,melee,scss-west-coast-34__scss-west-coast-singles,1007304,1007304,1129130,0,-1,"[""R3"", ""Round 3"", ""Round 3""]",Round Robin,1,F,5.0,[]
456435,gg__41949155,melee,scss-west-coast-34__scss-west-coast-singles,2510049,2510049,1129130,0,-1,"[""R4"", ""Round 4"", ""Round 4""]",Round Robin,1,G,5.0,[]
456436,gg__41949156,melee,scss-west-coast-34__scss-west-coast-singles,323132,323132,1030030,0,-1,"[""R4"", ""Round 4"", ""Round 4""]",Round Robin,1,H,5.0,[]
456437,gg__41949157,melee,scss-west-coast-34__scss-west-coast-singles,1030030,2510049,1030030,-1,0,"[""R5"", ""Round 5"", ""Round 5""]",Round Robin,1,I,5.0,[]


In [9]:
tournament_info

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,country,state,city,entrants,placings,losses,bracket_types,online,lat,lng
0,melee,mlg-world-finals-2015,MLG World Finals 2015,custom,mlg-world-finals-2015,,15,,1444953600,1445126400,US,LA,New Orleans,32,"[[1004, 1], [1003, 2], [16342, 3], [1028, 4], ...",{},b'{}',0,NaN,NaN
1,melee,httpsparagonchallongecomla_2015_melee_singles,Paragon Los Angeles 2015,challonge,https://paragon.challonge.com/la_2015_melee_si...,,15,,1441583146,1441657987,None,None,None,64,"[[1000, 1], [""Azen Zagenite"", 49], [1032, 17],...",{},b'{}',0,NaN,NaN
2,melee,httpsdl4-5challongecomdl45meleeprobracket,DrommeLAN4.5,challonge,https://dl4-5.challonge.com/DL45meleeProBracket,,15,,1430610920,1430626455,None,None,None,64,"[[6189, 1], [4465, 2], [12870, 4], [6103, 3], ...",{},b'{}',0,NaN,NaN
3,melee,httpsbeastsmashchallongecomb5msb,B.E.A.S.T 5,challonge,https://beastsmash.challonge.com/B5MSB,,15,,1420985773,1421110948,None,None,None,64,"[[1000, 3], [6189, 2], [4465, 1], [19573, 4], ...",{},b'{}',0,NaN,NaN
4,melee,httpsparagonchallongecomorlando_2015_melee_sin...,Paragon Orlando 2015,challonge,https://paragon.challonge.com/orlando_2015_mel...,,15,,1421621113,1421648276,None,None,None,66,"[[1004, 1], [4465, 3], [1003, 2], [6189, 5], [...",{},b'{}',0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6324,melee,the-foxy-horror-picture-show-a-gaylee-summit-q...,The Foxy Horror Picture Show: A Gaylee Summit ...,gg,the-foxy-horror-picture-show-a-gaylee-summit-q...,gaylee-singles,21,,1635717600,1635732000,None,None,None,40,"[[""215584"", 5], [""2512827"", 1], [""508872"", 7],...","{""215584"": [{""opponent"": ""225081"", ""bracket_or...","b'{""2"": 2, ""1"": 2}'",1,NaN,NaN
6325,melee,kalvar-cup-26-the-scary-one__melee-singles,"Kalvar Cup #26, the scary one",gg,kalvar-cup-26-the-scary-one,melee-singles,21,,1635719400,1635730200,None,None,None,15,"[[""618857"", 4], [""1918686"", 5], [""2221434"", 9]...","{""618857"": [{""opponent"": ""3936"", ""bracket_orde...","b'{""1"": 2}'",1,NaN,NaN
6326,melee,rock-chalk-jab-lock-85__melee-singles,Rock Chalk Jab Lock 85,gg,rock-chalk-jab-lock-85,melee-singles,21,,1635721200,1635739200,US,KS,Lawrence,7,"[[""71061"", 1], [""45269"", 4], [""1074815"", 2], [...","{""71061"": [], ""45269"": [{""opponent"": ""347099"",...","b'{""1"": 3}'",0,38.95742,-95.253958
6327,melee,scss-46__scss-singles,SCSS #46,gg,scss-46,scss-singles,21,,1635723000,1635739200,None,None,None,6,"[[""657464"", 1], [""2512542"", 6], [""2197580"", 5]...","{""657464"": [], ""2512542"": [{""opponent"": ""23148...","b'{""1"": 3}'",1,NaN,NaN


In [10]:
players['tag']

0                 -Caius-
1                Wizzrobe
2               Bladewise
3           GOL_482_JSalt
4                  RiSK__
               ...       
45537     Autumn from 207
45538       BasedVapeDuck
45539          Tron Bonne
45540    a bottle of soap
45541               Bongo
Name: tag, Length: 45542, dtype: object

In [11]:
#english unicode ascii xxx[\x00-\x7F]+xxx

In [12]:
players[players['c_country'] == 'US']

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,-Caius-,-Caius-,"[""-Caius-""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 2...","""""",None
1,melee,1028,Wizzrobe,"[""Wizzzrobe"", ""COG | Wizzrobe"", ""COG Wizzrobe""...","[""Envy"", ""ExG""]","{""twitter"": [""Wizzrobe""]}",United States,FL,,US,CA,Los Angeles,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 4...","{""melee/captainfalcon"": 659}",None
2,melee,1032,Bladewise,"[""Bladewise00"", ""[62-Bit] Bladewise"", ""Bladewi...","[""W2W""]","{""twitter"": [""bladewise00""]}",United States,WA,,US,CA,Seattle,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 9...","{""melee/peach"": 115}",None
3,melee,GOL_482_JSalt,GOL_482_JSalt,"[""GOL_482_JSalt""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 1...","""""",None
4,melee,RiSK__,RiSK__,"[""RiSK__""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 2...","""""",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45531,melee,2512734,Seven11,"[""Seven11""]",[],"{""twitter"": []}",United States,None,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None
45532,melee,2503795,hayden!,"[""hayden!""]","[""CNST""]","{""twitter"": []}",United States,NE,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None
45533,melee,2513297,Antwan,"[""Antwan""]",[],"{""twitter"": []}",None,None,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None
45534,melee,2513251,DBI,"[""DBI""]",[],"{""twitter"": []}",None,None,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None


In [13]:
(players['c_country'] == 'US').value_counts()

True     30110
False    15432
Name: c_country, dtype: int64

In [14]:
#((players['country'] == 'United States') or (players['country'] == 'US')).value_counts #TODO some have country us but don't have c_country. US in 'country' is either US or United States

In [15]:
us_tags_df = players[players['c_country'] == 'US']

In [16]:
us_tags_df

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,-Caius-,-Caius-,"[""-Caius-""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 2...","""""",None
1,melee,1028,Wizzrobe,"[""Wizzzrobe"", ""COG | Wizzrobe"", ""COG Wizzrobe""...","[""Envy"", ""ExG""]","{""twitter"": [""Wizzrobe""]}",United States,FL,,US,CA,Los Angeles,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 4...","{""melee/captainfalcon"": 659}",None
2,melee,1032,Bladewise,"[""Bladewise00"", ""[62-Bit] Bladewise"", ""Bladewi...","[""W2W""]","{""twitter"": [""bladewise00""]}",United States,WA,,US,CA,Seattle,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 9...","{""melee/peach"": 115}",None
3,melee,GOL_482_JSalt,GOL_482_JSalt,"[""GOL_482_JSalt""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 1...","""""",None
4,melee,RiSK__,RiSK__,"[""RiSK__""]",[],"{""twitter"": []}",,,,US,LA,New Orleans,"[{""key"": ""mlg-world-finals-2015"", ""placing"": 2...","""""",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45531,melee,2512734,Seven11,"[""Seven11""]",[],"{""twitter"": []}",United States,None,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None
45532,melee,2503795,hayden!,"[""hayden!""]","[""CNST""]","{""twitter"": []}",United States,NE,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None
45533,melee,2513297,Antwan,"[""Antwan""]",[],"{""twitter"": []}",None,None,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None
45534,melee,2513251,DBI,"[""DBI""]",[],"{""twitter"": []}",None,None,None,US,NE,Lincoln,"[{""key"": ""capitol-city-clash-2__melee-singles""...","""""",None


In [17]:
chars = ['mario', 'bowser', 'peach', 'yoshi', 'donkey kong', 'captain falcon', 'fox', 'ness', 
         'ice climbers', 'kirby', 'samus', 'zelda', 'link', 'pikachu', 'jigglypuff',
         'dr mario', 'luigi', 'ganondorf', 'falco', 'young link', 'pichu', 'mewtwo',
         'mr game and watch', 'marth', 'roy']

In [38]:
us_tags = us_tags_df['tag']

In [39]:

def hasCharacterNameRatio(tag):
    for char in chars:
        if fuzz.ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.ratio(char, tag)}")
            


In [48]:
us_tags.apply(hasCharacterNameRatio)

fox, fox 100


30110

In [41]:
def hasCharacterNamePartialRatio(tag):
    for char in chars:
        if fuzz.partial_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.partial_ratio(char, tag)}")

In [42]:
us_tags.apply(hasCharacterNamePartialRatio)

Wafox, fox 100
Cgl| cidfox, fox 100
Commonyoshi, yoshi 100
Hydro kirby, kirby 100
Grindfox, fox 100
Blue link, link 100
Gogglesthefox, fox 100
Slinky, link 100
Supadragonyoshi, yoshi 100
Agentfoxmulder, fox 100
Mlink, link 100
zeldafreak, zelda 100
newmariostar, mario 100
GU | timfox, fox 100
marthplayer69420, marth 100
Shiroyasha, roy 100
Business incorporated, ness 100
Ham_Destroyer, roy 100
Niceness, ness 100
ari, mario 100
ari, dr mario 100
McGinness, ness 100
k, donkey kong 100
k, kirby 100
k, link 100
k, pikachu 100
k, young link 100
link2213, link 100
Derilink, link 100
k2pichu, pichu 100
Leroy, roy 100
Destroyah, roy 100
kong, donkey kong 100
Iplayfalcon, falco 100
BlinkRider, link 100
Yummyness, ness 100
Hennessy, ness 100
Niceness, ness 100
Parcour_Robfox, fox 100
io, mario 100
io, dr mario 100
roygbiv, roy 100
Greenmario, mario 100
FinesseQuik, ness 100
s-royal, roy 100
Umarth, marth 100
witness, ness 100
Unorthodoxfox, fox 100
Troy, roy 100
Blink, link 100
Viceroy, roy 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [43]:
def hasCharacterNameTokenSortRatio(tag):
    for char in chars:
        if fuzz.token_sort_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_sort_ratio(char, tag)}")

In [44]:
us_tags.apply(hasCharacterNameTokenSortRatio)

Yoshi, yoshi 100
Fox, fox 100
Bowser, bowser 100
Roy, roy 100
Yoshi, yoshi 100
Yoshi, yoshi 100
Marth$, marth 100
Mario, mario 100
Fox, fox 100
Ness?, ness 100
Roy, roy 100
Falco, falco 100
Kirby, kirby 100
Ganondorf, ganondorf 100
Donkey Kong, donkey kong 100
Falco, falco 100
Ness, ness 100
Roy, roy 100
Link, link 100
Yoshi, yoshi 100
Mewtwo, mewtwo 100
Marth, marth 100
Luigi, luigi 100
Fox, fox 100
Kirby, kirby 100
Young Link, young link 100
Jigglypuff, jigglypuff 100
Dr. Mario, dr mario 100
Bowser, bowser 100
Peach, peach 100
Ice Climbers, ice climbers 100
Samus, samus 100
Mario, mario 100
Captain Falcon, captain falcon 100
Pichu, pichu 100
Pikachu, pikachu 100
Mario, mario 100
Mario, mario 100
fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [49]:
def hasCharacterNameTokenSetRatio(tag):
    for char in chars:
        if fuzz.token_set_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_set_ratio(char, tag)}")

In [50]:
us_tags.apply(hasCharacterNameTokenSetRatio) #BEST

Ice, ice climbers 100
Yoshi, yoshi 100
Luigi master tae/taciouse, luigi 100
Hydro kirby, kirby 100
Blue link, link 100
Fox, fox 100
Luigi Ka-Master, luigi 100
FB | Flamin Roy, roy 100
Dark Blue Yoshi, yoshi 100
Luigi master tae -lmttaciouse, luigi 100
Grey-Fox, fox 100
Dr. Mario World Champion, mario 100
Dr. Mario World Champion, dr mario 100
The Last Dope Marth, marth 100
Young, young link 100
kong, donkey kong 100
Peachy Pichu, pichu 100
Flamin Roy, roy 100
Hung Link, link 100
Bowser, bowser 100
Grey-Fox, fox 100
Peach Shampoo, peach 100
Woke Fox, fox 100
big peach, peach 100
DP's Mario, mario 100
a Link named SlykBacc, link 100
Young $, young link 100
Deep Fox, fox 100
Roy, roy 100
Marth Vader, marth 100
Yoshi, yoshi 100
Luigi Board, luigi 100
luigi master tae, luigi 100
Link Master 3000, link 100
Yoshi, yoshi 100
L R A Start Fox, fox 100
Marth Vader, marth 100
Nick han Tekken marth MW2, marth 100
DonKey, donkey kong 100
Captain Falco, falco 100
Young, young link 100
Young$, young l

0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [51]:
us_tags = us_tags_df['tag'].str.replace(r'\W', '') #remove special characters

/var/folders/ns/jhsc1w3d101cp62xgg0jvvg00000gn/T/ipykernel_95186/2677058677.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  us_tags = us_tags_df['tag'].str.replace(r'\W', '') #remove special characters


In [52]:
us_tags

0                Caius
1             Wizzrobe
2            Bladewise
3        GOL_482_JSalt
4               RiSK__
             ...      
45531          Seven11
45532           hayden
45533           Antwan
45534              DBI
45539        TronBonne
Name: tag, Length: 30110, dtype: object

In [53]:
def hasCharacterNameRatio(tag): 
    for char in chars:
        if fuzz.ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.ratio(char, tag)}")

In [54]:
us_tags.apply(hasCharacterNameRatio)

fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [55]:
def hasCharacterNamePartialRatio(tag):
    for char in chars:
        if fuzz.partial_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.partial_ratio(char, tag)}")

In [56]:
us_tags.apply(hasCharacterNamePartialRatio)

Wafox, fox 100
Cglcidfox, fox 100
Commonyoshi, yoshi 100
Hydrokirby, kirby 100
Grindfox, fox 100
Bluelink, link 100
Gogglesthefox, fox 100
Slinky, link 100
Supadragonyoshi, yoshi 100
Agentfoxmulder, fox 100
Mlink, link 100
zeldafreak, zelda 100
newmariostar, mario 100
GUtimfox, fox 100
marthplayer69420, marth 100
Shiroyasha, roy 100
Businessincorporated, ness 100
Ham_Destroyer, roy 100
Niceness, ness 100
ari, mario 100
ari, dr mario 100
McGinness, ness 100
k, donkey kong 100
k, kirby 100
k, link 100
k, pikachu 100
k, young link 100
link2213, link 100
Derilink, link 100
k2pichu, pichu 100
Leroy, roy 100
Destroyah, roy 100
kong, donkey kong 100
Iplayfalcon, falco 100
BlinkRider, link 100
Yummyness, ness 100
Hennessy, ness 100
Niceness, ness 100
Parcour_Robfox, fox 100
io, mario 100
io, dr mario 100
roygbiv, roy 100
Greenmario, mario 100
FinesseQuik, ness 100
sroyal, roy 100
Umarth, marth 100
witness, ness 100
Unorthodoxfox, fox 100
Troy, roy 100
Blink, link 100
Viceroy, roy 100
Mistyfoxy

0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [57]:
def hasCharacterNameTokenSortRatio(tag):
    for char in chars:
        if fuzz.token_sort_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_sort_ratio(char, tag)}")

In [58]:
us_tags.apply(hasCharacterNameTokenSortRatio)

Yoshi, yoshi 100
Fox, fox 100
Bowser, bowser 100
Roy, roy 100
Yoshi, yoshi 100
Yoshi, yoshi 100
Marth, marth 100
Mario, mario 100
Fox, fox 100
Ness, ness 100
Roy, roy 100
Falco, falco 100
Kirby, kirby 100
Ganondorf, ganondorf 100
Falco, falco 100
Ness, ness 100
Roy, roy 100
Link, link 100
Yoshi, yoshi 100
Mewtwo, mewtwo 100
Marth, marth 100
Luigi, luigi 100
Fox, fox 100
Kirby, kirby 100
Jigglypuff, jigglypuff 100
Bowser, bowser 100
Peach, peach 100
Samus, samus 100
Mario, mario 100
Pichu, pichu 100
Pikachu, pikachu 100
Mario, mario 100
Mario, mario 100
fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [59]:
def hasCharacterNameTokenSetRatio(tag):
    for char in chars:
        if fuzz.token_set_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_set_ratio(char, tag)}")

In [60]:
us_tags.apply(hasCharacterNameTokenSetRatio)

Ice, ice climbers 100
Yoshi, yoshi 100
Fox, fox 100
Young, young link 100
kong, donkey kong 100
Bowser, bowser 100
Young, young link 100
Roy, roy 100
Yoshi, yoshi 100
Yoshi, yoshi 100
DonKey, donkey kong 100
Young, young link 100
Young, young link 100
Mr, mr game and watch 100
Marth, marth 100
game, mr game and watch 100
Young, young link 100
Captain, captain falcon 100
Mario, mario 100
Mario, dr mario 100
Game, mr game and watch 100
watch, mr game and watch 100
Fox, fox 100
Game, mr game and watch 100
Ice, ice climbers 100
Ness, ness 100
Roy, roy 100
Falco, falco 100
Kirby, kirby 100
Mr, mr game and watch 100
Ganondorf, ganondorf 100
Falco, falco 100
Ness, ness 100
Roy, roy 100
Link, link 100
Link, young link 100
Yoshi, yoshi 100
Mewtwo, mewtwo 100
Marth, marth 100
Luigi, luigi 100
Fox, fox 100
Kirby, kirby 100
Jigglypuff, jigglypuff 100
Bowser, bowser 100
Peach, peach 100
Samus, samus 100
Mario, mario 100
Mario, dr mario 100
Pichu, pichu 100
Pikachu, pikachu 100
Mario, mario 100
Mari

0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [61]:
us_tags = us_tags_df['tag'].str.replace(r'\W', '').str.normalize('NFKD') #remove special characters and normalize

/var/folders/ns/jhsc1w3d101cp62xgg0jvvg00000gn/T/ipykernel_95186/1775032349.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  us_tags = us_tags_df['tag'].str.replace(r'\W', '').str.normalize('NFKD') #remove special characters and normalize


In [62]:
us_tags

0                Caius
1             Wizzrobe
2            Bladewise
3        GOL_482_JSalt
4               RiSK__
             ...      
45531          Seven11
45532           hayden
45533           Antwan
45534              DBI
45539        TronBonne
Name: tag, Length: 30110, dtype: object

In [63]:
def hasCharacterNameRatio(tag):
    for char in chars:
        if fuzz.ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.ratio(char, tag)}")

In [64]:
us_tags.apply(hasCharacterNameRatio)

fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [65]:
def hasCharacterNamePartialRatio(tag):
    for char in chars:
        if fuzz.partial_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.partial_ratio(char, tag)}")

In [66]:
us_tags.apply(hasCharacterNamePartialRatio)

Wafox, fox 100
Cglcidfox, fox 100
Commonyoshi, yoshi 100
Hydrokirby, kirby 100
Grindfox, fox 100
Bluelink, link 100
Gogglesthefox, fox 100
Slinky, link 100
Supadragonyoshi, yoshi 100
Agentfoxmulder, fox 100
Mlink, link 100
zeldafreak, zelda 100
newmariostar, mario 100
GUtimfox, fox 100
marthplayer69420, marth 100
Shiroyasha, roy 100
Businessincorporated, ness 100
Ham_Destroyer, roy 100
Niceness, ness 100
ari, mario 100
ari, dr mario 100
McGinness, ness 100
k, donkey kong 100
k, kirby 100
k, link 100
k, pikachu 100
k, young link 100
link2213, link 100
Derilink, link 100
k2pichu, pichu 100
Leroy, roy 100
Destroyah, roy 100
kong, donkey kong 100
Iplayfalcon, falco 100
BlinkRider, link 100
Yummyness, ness 100
Hennessy, ness 100
Niceness, ness 100
Parcour_Robfox, fox 100
io, mario 100
io, dr mario 100
roygbiv, roy 100
Greenmario, mario 100
FinesseQuik, ness 100
sroyal, roy 100
Umarth, marth 100
witness, ness 100
Unorthodoxfox, fox 100
Troy, roy 100
Blink, link 100
Viceroy, roy 100
Mistyfoxy

0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [67]:
def hasCharacterNameTokenSortRatio(tag):
    for char in chars:
        if fuzz.token_sort_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_sort_ratio(char, tag)}")

In [68]:
us_tags.apply(hasCharacterNameTokenSortRatio)

Yoshi, yoshi 100
Fox, fox 100
Bowser, bowser 100
Roy, roy 100
Yoshi, yoshi 100
Yoshi, yoshi 100
Marth, marth 100
Mario, mario 100
Fox, fox 100
Ness, ness 100
Roy, roy 100
Falco, falco 100
Kirby, kirby 100
Ganondorf, ganondorf 100
Falco, falco 100
Ness, ness 100
Roy, roy 100
Link, link 100
Yoshi, yoshi 100
Mewtwo, mewtwo 100
Marth, marth 100
Luigi, luigi 100
Fox, fox 100
Kirby, kirby 100
Jigglypuff, jigglypuff 100
Bowser, bowser 100
Peach, peach 100
Samus, samus 100
Mario, mario 100
Pichu, pichu 100
Pikachu, pikachu 100
Mario, mario 100
Mario, mario 100
fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [69]:
def hasCharacterNameTokenSetRatio(tag):
    for char in chars:
        if fuzz.token_set_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_set_ratio(char, tag)}")

In [70]:
us_tags.apply(hasCharacterNameTokenSetRatio)

Ice, ice climbers 100
Yoshi, yoshi 100
Fox, fox 100
Young, young link 100
kong, donkey kong 100
Bowser, bowser 100
Young, young link 100
Roy, roy 100
Yoshi, yoshi 100
Yoshi, yoshi 100
DonKey, donkey kong 100
Young, young link 100
Young, young link 100
Mr, mr game and watch 100
Marth, marth 100
game, mr game and watch 100
Young, young link 100
Captain, captain falcon 100
Mario, mario 100
Mario, dr mario 100
Game, mr game and watch 100
watch, mr game and watch 100
Fox, fox 100
Game, mr game and watch 100
Ice, ice climbers 100
Ness, ness 100
Roy, roy 100
Falco, falco 100
Kirby, kirby 100
Mr, mr game and watch 100
Ganondorf, ganondorf 100
Falco, falco 100
Ness, ness 100
Roy, roy 100
Link, link 100
Link, young link 100
Yoshi, yoshi 100
Mewtwo, mewtwo 100
Marth, marth 100
Luigi, luigi 100
Fox, fox 100
Kirby, kirby 100
Jigglypuff, jigglypuff 100
Bowser, bowser 100
Peach, peach 100
Samus, samus 100
Mario, mario 100
Mario, dr mario 100
Pichu, pichu 100
Pikachu, pikachu 100
Mario, mario 100
Mari

0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [81]:
#us_tags = us_tags_df['tag'].str.lower().str.replace(r'\W', '').str.normalize('NFKD') #remove special characters, normalize, and put into lowercase
us_tags = us_tags_df['tag'].str.lower().str.normalize('NFKD') #normalize, and put into lowercase


In [82]:
us_tags

0              -caius-
1             wizzrobe
2            bladewise
3        gol_482_jsalt
4               risk__
             ...      
45531          seven11
45532          hayden!
45533           antwan
45534              dbi
45539       tron bonne
Name: tag, Length: 30110, dtype: object

In [83]:
def hasCharacterNameRatio(tag):
    for char in chars:
        if fuzz.ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.ratio(char, tag)}")

In [84]:
us_tags.apply(hasCharacterNameRatio)

yoshi, yoshi 100
fox, fox 100
bowser, bowser 100
roy, roy 100
yoshi, yoshi 100
yoshi, yoshi 100
mario, mario 100
fox, fox 100
roy, roy 100
falco, falco 100
kirby, kirby 100
ganondorf, ganondorf 100
donkey kong, donkey kong 100
falco, falco 100
ness, ness 100
roy, roy 100
link, link 100
yoshi, yoshi 100
mewtwo, mewtwo 100
marth, marth 100
luigi, luigi 100
fox, fox 100
kirby, kirby 100
young link, young link 100
jigglypuff, jigglypuff 100
bowser, bowser 100
peach, peach 100
ice climbers, ice climbers 100
samus, samus 100
mario, mario 100
captain falcon, captain falcon 100
pichu, pichu 100
pikachu, pikachu 100
mario, mario 100
mario, mario 100
fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [85]:
def hasCharacterNamePartialRatio(tag):
    for char in chars:
        if fuzz.partial_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.partial_ratio(char, tag)}")

In [86]:
us_tags.apply(hasCharacterNamePartialRatio)

l, captain falcon 100
l, ice climbers 100
l, zelda 100
l, link 100
l, jigglypuff 100
l, luigi 100
l, falco 100
l, young link 100
ice, ice climbers 100
kirbykaze, kirby 100
peachfuzz, peach 100
falcoty, falco 100
wafox, fox 100
cgl| cidfox, fox 100
lc, captain falcon 100
lc, falco 100
yoshi, yoshi 100
luigi master tae/taciouse, luigi 100
hi, yoshi 100
peaches, peach 100
commonyoshi, yoshi 100
hydro kirby, kirby 100
grindfox, fox 100
blue link, link 100
mari, mario 100
mari, dr mario 100
yoshido, yoshi 100
luigigoshard, luigi 100
falcomist, falco 100
ji, jigglypuff 100
fox, fox 100
kirbyfk, kirby 100
foxyfaux, fox 100
linko, link 100
gogglesthefox, fox 100
zeldafreak, zelda 100
tc, mr game and watch 100
slinky, link 100
ai, captain falcon 100
foxy grampa, fox 100
supadragonyoshi, yoshi 100
agentfoxmulder, fox 100
mlink, link 100
puff, jigglypuff 100
luigi ka-master, luigi 100
s-royal, roy 100
rsc|wafox, fox 100
zeldafreak, zelda 100
fb | flamin roy, roy 100
dark blue yoshi, yoshi 100
fox

KeyboardInterrupt: 

In [87]:
def hasCharacterNameTokenSortRatio(tag):
    for char in chars:
        if fuzz.token_sort_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_sort_ratio(char, tag)}")

In [88]:
us_tags.apply(hasCharacterNameTokenSortRatio)

yoshi, yoshi 100
fox, fox 100
bowser, bowser 100
roy, roy 100
yoshi, yoshi 100
yoshi, yoshi 100
marth$, marth 100
mario, mario 100
fox, fox 100
ness?, ness 100
roy, roy 100
falco, falco 100
kirby, kirby 100
ganondorf, ganondorf 100
donkey kong, donkey kong 100
falco, falco 100
ness, ness 100
roy, roy 100
link, link 100
yoshi, yoshi 100
mewtwo, mewtwo 100
marth, marth 100
luigi, luigi 100
fox, fox 100
kirby, kirby 100
young link, young link 100
jigglypuff, jigglypuff 100
dr. mario, dr mario 100
bowser, bowser 100
peach, peach 100
ice climbers, ice climbers 100
samus, samus 100
mario, mario 100
captain falcon, captain falcon 100
pichu, pichu 100
pikachu, pikachu 100
mario, mario 100
mario, mario 100
fox, fox 100


0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object

In [89]:
def hasCharacterNameTokenSetRatio(tag):
    for char in chars:
        if fuzz.token_set_ratio(char, tag) == 100:
            print(f"{tag}, {char} {fuzz.token_set_ratio(char, tag)}")

In [90]:
us_tags.apply(hasCharacterNameTokenSetRatio)

ice, ice climbers 100
yoshi, yoshi 100
luigi master tae/taciouse, luigi 100
hydro kirby, kirby 100
blue link, link 100
fox, fox 100
luigi ka-master, luigi 100
fb | flamin roy, roy 100
dark blue yoshi, yoshi 100
luigi master tae -lmttaciouse, luigi 100
grey-fox, fox 100
dr. mario world champion, mario 100
dr. mario world champion, dr mario 100
the last dope marth, marth 100
young, young link 100
kong, donkey kong 100
peachy pichu, pichu 100
flamin roy, roy 100
hung link, link 100
bowser, bowser 100
grey-fox, fox 100
peach shampoo, peach 100
woke fox, fox 100
big peach, peach 100
dp's mario, mario 100
a link named slykbacc, link 100
young $, young link 100
deep fox, fox 100
roy, roy 100
marth vader, marth 100
yoshi, yoshi 100
luigi board, luigi 100
luigi master tae, luigi 100
link master 3000, link 100
yoshi, yoshi 100
l r a start fox, fox 100
marth vader, marth 100
nick han tekken marth mw2, marth 100
donkey, donkey kong 100
captain falco, falco 100
young, young link 100
young$, young l

0        None
1        None
2        None
3        None
4        None
         ... 
45531    None
45532    None
45533    None
45534    None
45539    None
Name: tag, Length: 30110, dtype: object